# Bond Valuation
## Lecture Notes by Juan Carlo F. Mallari (jmallari@ateneo.edu)
MATH 100.2: Topics in Financial Mathematics II \ First Semester, S.Y. 2021-2022 \ Ateneo de Manila University

In [4]:
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta

print("packages")

packages


In [5]:
def get_tenor(
    start_date, end_date
    ):
    """
    Calculates the tenor in years from start_date to end_date following the Actual/360 day count convention
    
    Inputs:
    start_date: start date in "yyyy-mm-dd" format (usually today)
    end_date: end date in "yyyy-mm-dd" format (coupon payment/maturity date)
    """
    
    day_count = (end_date - start_date).days
    tenor = day_count / 360.0
    
    return tenor

print("tenor function")

tenor function


In [6]:
def get_bond_price_using_yield(
    start_date, maturity_date, par_value, coupon_rate, coupon_freq, ytm
    ):
    """
    Calculates the (clean) price of a (plain vanilla) coupon-bearing bond
    
    Notes:
    1) The Actual/360 day count convention is used.
    2) The yield is assumed to be continuously compounded.
    
    Inputs:
    start_date: start date in "yyyy-mm-dd" format (usually today)
    maturity_date: maturity date in "yyyy-mm-dd" format
    par_value: par value of the bond
    coupon_rate: coupon rate in decimal form
    coupon_freq: number of coupon payments every year (1, 2, 4, 12)
    ytm: yield to maturity
    """
    coupon = (coupon_rate * par_value) / coupon_freq
    
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    maturity_date = dt.datetime.strptime(maturity_date, '%Y-%m-%d') 
    
    # Create list of coupon dates
    coupon_dates = []
    coupon_date = maturity_date
    while start_date < coupon_date:
        coupon_dates.append(coupon_date)
        coupon_date -= relativedelta(months = 12 / coupon_freq)
    
    coupon_dates = coupon_dates[::-1] # rearrange dates
    
    # Create list of time values
    times = [get_tenor(start_date, t) for t in coupon_dates]
    
    # Create list of coupons
    coupon = coupon_rate * par_value
    coupons = [coupon] * (len(coupon_dates) - 1)
    coupons += [coupon + par_value]
    
    # Create list of discount factors
    discount_factors = [np.exp(-ytm * t) for t in times]
    
    df = pd.DataFrame({'coupon_dates' : coupon_dates})
    df.loc[:, "t"] = times
    df.loc[:, "CF"] = coupons
    df.loc[:, "DF"] = discount_factors
    df.loc[:, "PV"] = df["CF"] * df["DF"]
    
    price = round(df["PV"].sum(), 2)
    
    return df, price

print("price function")

price function


## Example
Suppose today is October 12, 2021. A 10-year corporate bond has $1000 par value, 10\% coupon bond that pays interest annually, with maturity date on December 15, 2025. Suppose that the prevailing market yield for the bond is 11\% compounded continuously. Assume an actual/360 day-count convention.

What is the current price of the bond?

In [7]:
df, price = get_bond_price_using_yield(
    start_date = "2021-10-12", 
    maturity_date = "2025-12-15", 
    par_value = 1000, 
    coupon_rate = 0.10, 
    coupon_freq = 1,
    ytm = 0.11
)

print("price calculation")

price calculation


In [104]:
df

,coupon_dates,t,CF,DF,PV
0,2021-12-15,0.177778,100.0,0.980634,98.063441
1,2022-12-15,1.191667,100.0,0.877145,87.714468
2,2023-12-15,2.205556,100.0,0.784577,78.457657
3,2024-12-15,3.222222,100.0,0.701563,70.156309
4,2025-12-15,4.236111,1100.0,0.627525,690.277187


In [105]:
price

1024.67

In [17]:
x = [1,2]
y = [0.1025, 0.1033]

# Rate for t_2
t = df.iloc[1, 1]

print(np.interp(t, x, y))

# Rate for t_1
t = df.iloc[0, 1]

print(np.interp(t, x, y))

0.10265333333333333
0.1025


In [13]:
def linear_interp(t,x,y):
    '''
    Linearly interpolates the zero rate for tenor t 
    using the equation of line determined by the two available points 
    (x_1,y_1) and (x_2,y_2) whose tenors x_1 and x_2 are nearest to t
    
    -------
    Inputs:
    t = tenor of the unknown zero rate 
    x = list [x_1,x_2]
    y = list [y_1,y_2]
    '''
    m = (y[1]-y[0])/(x[1]-x[0])
    
    y = y[0] + m*(t-x[0])
    
    return y

print("linear interpolation function")

linear interpolation function


In [14]:
x = [1,2]
y = [0.1025, 0.1033]

# Rate for t_2
t = df.iloc[1, 1]

print(linear_interp(t, x, y))

# Rate for t_1
t = df.iloc[0, 1]

print(linear_interp(t, x, y))

0.10265333333333333
0.10184222222222221
